In [2]:
import contractions
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /home/copper_turtle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/copper_turtle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/copper_turtle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv("tweet_data.csv")

In [4]:

def replace_retweet(tweet, default_replace=""):
  tweet = re.sub("^RT\s+", default_replace, tweet)
  return tweet

def replace_user(tweet, default_replace="twitteruser"):
  tweet = re.sub("\B@\w+", default_replace, tweet)
  return tweet

def replace_url(tweet, default_replace=""):
  tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)
  return tweet

def replace_hashtag(tweet, default_replace=""):
  tweet = re.sub('#+', default_replace, tweet)
  return tweet

def to_lowercase(tweet):
  return tweet.lower()

def word_repetition(tweet):
  tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
  return tweet

def punct_repetition(tweet, default_replace=""):
  tweet = re.sub(r'[\?\.\!]+(?=[\?\.!])', default_replace, tweet)
  return tweet

def fix_contractions(tweet):
  return contractions.fix(tweet)

def tokenize(tweet):
  tokens = word_tokenize(tweet)
  return tokens

def demojize(tweet):
    tweet = emoji.demojize(tweet)
    return tweet



<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\B'
<>:10: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\B'
<>:10: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipykernel_85215/2047234332.py:2: SyntaxWarning: invalid escape sequence '\s'
  tweet = re.sub("^RT\s+", default_replace, tweet)
/tmp/ipykernel_85215/2047234332.py:6: SyntaxWarning: invalid escape sequence '\B'
  tweet = re.sub("\B@\w+", default_replace, tweet)
/tmp/ipykernel_85215/2047234332.py:10: SyntaxWarning: invalid escape sequence '\/'
  tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)


In [5]:
def custom_tokenize(tweet, keep_punct = False, keep_alnum = False, keep_stop = False):
    token_list = word_tokenize(tweet)
    
    if not keep_punct:
        token_list = [token for token in token_list if token not in string.punctuation]
        
    if not keep_alnum:
        token_list = [token for token in token_list if token.isalpha()]
    
    if not keep_stop:
        stop_words = set(stopwords.words('english'))
        stop_words.discard('not')
        token_list = [token for token in token_list if not token in stop_words]
        
          
    
    return token_list

In [6]:
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
snowball_stemmer = SnowballStemmer('english')

In [7]:
def stem_tokens(tokens, stemmer):
    token_list = []
    for token in tokens:
        token_list.append(stemmer.stem(token))
    return token_list

In [25]:
def process_tweet(tweet, verbose=False):
  # if verbose: print("Initial tweet: {}".format(tweet))
  # print(tweet)



  ## Twitter Features
    # replace retweet
  tweet = replace_retweet(tweet, default_replace="")
  # replace user tag
  tweet = replace_user(tweet, default_replace="twitteruser")
  # replace url
  tweet = replace_url(tweet, default_replace="")
  # replace hashtag
  tweet = replace_hashtag(tweet, default_replace="")
  if verbose: print("Post Twitter processing tweet: {}".format(tweet))

  ## Word Features
    # lower case
  tweet = to_lowercase(tweet)
    # replace contractions
  tweet = fix_contractions(tweet)
    # replace punctuation repetition
  tweet = punct_repetition(tweet, default_replace="")
    # replace word repetition
  tweet = word_repetition(tweet)
    # replace emojis
  tweet = demojize(tweet)
  if verbose: print("Post Word processing tweet: {}".format(tweet))

  ## Tokenization & Stemming
    # tokenize
  tokens = custom_tokenize(tweet, keep_alnum=True)
    # define stemmer
    # stem tokens
  stem = stem_tokens(tokens, SnowballStemmer("english"))
  return stem

In [26]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'the', 'wasn', 'couldn', "should've", 'while', 'you', 'that', "couldn't", 'ours', 'who', 'on', 'theirs', 'having', "don't", "shouldn't", 'they', 'then', 'herself', 'just', "doesn't", 'before', 'isn', "isn't", 'haven', 'myself', 'itself', 'his', 'because', 'same', 'their', 'once', 'we', "you've", 'o', "mightn't", 'what', 'shouldn', 'themselves', "you'll", "won't", 'has', 'below', 'to', 'me', 'during', 'own', 'are', 'off', 'more', 'y', 'here', 'out', 'ain', 'hers', 'weren', 'i', 'be', 'about', 'he', 'through', 'nor', 'too', "didn't", 'didn', 'which', 'down', 'yours', 'up', 'both', 'after', 'there', 'with', 'these', 'any', 'but', 're', 'doesn', "needn't", 'been', 'being', 'hasn', 'will', 'only', 'against', 'most', 'over', 'won', 'this', 'why', 'very', 'of', 'under', 'himself', 'have', 'can', 'such', 'my', 've', 'again', 'between', 'hadn', 'should', "you'd", 'from', 'yourself', "mustn't", 'in', 'ourselves', 'how', 'him', 'don', "hadn't", 't', 'each', "haven't", 'mustn', 'yourselves', 'whe

In [27]:
df["tokens"] = df["tweet_text"].apply(process_tweet)
df["tweet_sentiment"] = df["sentiment"].apply(lambda i: 1
                                              if i == 'positive' else 0)

In [28]:
df

,textID,tweet_text,sentiment,tokens,tweet_sentiment
0,1956967666,Layin n bed with a headache ughhhh...waitin o...,negative,"[layin, n, bed, headach, ughh.waitin, call]",0
1,1956967696,Funeral ceremony...gloomy friday...,negative,"[funer, ceremony.gloomi, friday]",0
2,1956967789,wants to hang out with friends SOON!,positive,"[want, hang, friend, soon]",1
3,1956968477,Re-pinging @ghostridah14: why didn't you go to...,negative,"[re-ping, twitterus, not, go, prom, bf, not, l...",0
4,1956968636,Hmmm. http://www.djhero.com/ is down,negative,[hmm],0
...,...,...,...,...,...
18722,1753918818,"had SUCH and AMAZING time last night, McFly we...",positive,"[amaz, time, last, night, mcfli, incred]",1
18723,1753918881,@jasimmo Ooo showing of your French skills!! l...,positive,"[twitterus, oo, show, french, skill, lol, thin...",1
18724,1753918900,Succesfully following Tayla!!,positive,"[succes, follow, tayla]",1
18725,1753919001,Happy Mothers Day All my love,positive,"[happi, mother, day, love]",1


In [15]:
X = df["tokens"]
y = df["sentiment"]

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

Training Data Shape: (12547,)
Testing Data Shape:  (6180,)


In [20]:
count_vect = CountVectorizer()

In [22]:
# count_vect.fit(X_train)
# X_train_counts = count_vect.transform(X_train)

count_vect.fit_transform(X_train)

AttributeError: 'list' object has no attribute 'lower'